# Lidando com Valores Ausentes na biblioteca pandas

**Valores Ausentes** ocorrem quando não foram informados valores para certas variáveis em um dataset. É comum ocorrer quando utilizamos dados reais.

O que é um valor ausente para o Pandas?
* None (Nativo do Python)
* NaN (Not a Number): constante da biblioteca Numpy

Neste notebook vamos aprender algumas maneiras de lidar com valores ausentes.

Link para a documentação do Pandas sobre missing data: https://pandas.pydata.org/docs/user_guide/missing_data.html

## Instalação e Importação da biblioteca pandas

Antes de rodar este notebook, garanta que a biblioteca abaixo estejam instaladas em seu ambiente:

In [4]:
# conda install pandas

In [5]:
import pandas as pd

## Criação do dataframe e análise inicial dos dados

Nosso dataset de testes será o `titanic.csv`, um dataset bastante famoso em ciência de dados. Esse dataset contém as seguintes informações sobre os passageiros do Titanic:

* `PassengerId` = ID do passageiro do navio (chave primária).
* `Pclass	` = Tipo de classe de passagem (valor numérico entre 1 e 3), sendo 1 a melhor classe e 3 a pior classe.
* `Name` = Nome do passageiro
* `Sex` = Gênero do passageiro, com valores male/female.
* `Age` = Idade do passageiro na data da ocorrência do naufrágio (em anos).
* `SibSp` = Número de irmãos / cônjuges a bordo.
* `Parch` = Número de pais / filhos a bordo.
* `Ticket` = Número do ticket.
* `Fare` = Valor da passagem.
* `Cabin` = Código de identificação da Cabine.
* `Embarked` = Porto ondem o passageiro embarcou no navio. (C = Cherbourg, Q = Queenstown, S = Southampton)
* `survived` = Se sobreviveu ao naufrágio estará como 1 e caso esteja com 0 (zero) não sobreviveu.


In [6]:
# Carrega o dataset
dados = pd.read_csv('titanic.csv')

# Exibe as primeiras linhas do dataset
dados.head()

FileNotFoundError: [Errno 2] No such file or directory: 'titanic.csv'

In [ ]:
# Outra forma de observar os dados do dataset é com a função sample
dados.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
254,255,0,3,"Rosblom, Mrs. Viktor (Helena Wilhelmina)",female,41.0,0,2,370129,20.2125,NaN,S
51,52,0,3,"Nosworthy, Mr. Richard Cater",male,21.0,0,0,A/4. 39886,7.8000,NaN,S
636,637,0,3,"Leinonen, Mr. Antti Gustaf",male,32.0,0,0,STON/O 2. 3101292,7.9250,NaN,S
552,553,0,3,"O'Brien, Mr. Timothy",male,NaN,0,0,330979,7.8292,NaN,Q
380,381,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,NaN,C


Podemos obter algumas estatísticas sobre os dados usando a função `describe`

In [ ]:
dados.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Exercício 1**: Com a execução da célula anterior, podemos notar que o atributo `Age` possui um valor máximo de 80. Esse valor pode ser considerado um outlier? Como você poderia justificar sua afirmação (com base no dataset e com base em informações externas)?

#### Resposta:

**Sim**, pode ser considerado um outlier, pelos seguintes motivos: 

**75% do dataset tem idade <= 38**, é uma diferença muito grande para o valor máximo. e tem o cálculo de IQR:
- Q1 (25%): 20.125
- Q3 (75%): 38.000
- IQR = Q3 - Q1 = 17.875
- Limite superior = Q3+(1.5*IQR) = 38+(1.5*17.875) = 64.81 (aprox) 

com isso, pode ser dito que qualquer idade acima de 64.81 podem ser consideradas outliers estatisticamente.

Agora, baseado em informações externas:
- Google:
**"Na primeira metade do século XX a esperança de vida passou de 32 anos em 1900 para 34,1 anos em 1913 e deu um salto para 45,7 anos em 1950"**

- ChatGPT: **A maioria dos passageiros do Titanic eram adultos jovens, trabalhadores ou membros de famílias viajando. Pessoas com 80 anos ou mais eram bastante raras no início do século XX, especialmente em viagens longas como essa. A expectativa de vida no início do século XX era muito mais baixa do que hoje, o que corrobora o fato de que uma pessoa com 80 anos naquela época era uma exceção.** 

## Identificando valores ausentes

A função `isnull()` retorna verdadeiro se um valor em uma posição do dataframe é vazio

In [ ]:
dados['Age'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Name: Age, Length: 891, dtype: bool

Podemos obter a contagem de valores nulos em cada coluna em um novo dataframe

In [ ]:
contagem_nulos = dados.isnull().sum()
contagem_nulos

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Exclusão de Registros com dados faltantes

Uma forma de lidar com valores ausentes é excluir do dataframe as linhas que possuem valores ausentes. Isso pode ser feito facilmente através da função `dropna`. Essa função retorna um dataset sem as linhas que possuem algum valor ausente


In [ ]:
sem_nulos = dados.dropna()

In [ ]:
sem_nulos

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


**EXERCÍCIO 2**: Quantos registros foram excluídos do dataset?

In [ ]:
num_linhas_original = len(dados)
num_linhas_sem_nulos = len(sem_nulos)
qtd_itens_apagados= num_linhas_original - num_linhas_sem_nulos

print(f"Número de linhas no dataset original: {num_linhas_original}")
print(f"Número de linhas no dataset sem nulos: {num_linhas_sem_nulos}")
print(f"Número de linhas apagadas: {qtd_itens_apagados}")

Número de linhas no dataset original: 891
Número de linhas no dataset sem nulos: 183
Número de linhas apagadas: 708


**EXERCÍCIO 3**: Pode-se também excluir do dataset as colunas que contém valores faltantes. Para isso deve-se informar o parâmetro `axis=1` para a função `dropna`. Teste essa função e verifique quantas colunas foram excluídas do dataframe.

In [ ]:
colunas_sem_nulos = dados.dropna(axis=1)

num_colunas_original = len(dados.columns)
num_colunas_sem_nulos = len(colunas_sem_nulos.columns)
qtd_colunas_apagadas = num_colunas_original - num_colunas_sem_nulos

print(f"Número de colunas no dataset original: {num_colunas_original}")
print(f"Número de colunas no dataset sem nulos: {num_colunas_sem_nulos}")
print(f"Número de colunas apagadas: {qtd_colunas_apagadas}")

Número de colunas no dataset original: 12
Número de colunas no dataset sem nulos: 9
Número de colunas apagadas: 3


## Inputação de Dados

### Substituição Dummy

Outra forma de lidar com valores ausentes é preencher os valores faltantes com um valor padrão. A função `fillna(valor)` pode ser utilizada para realizar esta tarefa.

*Tome cuidado*: a função `fillna` preenche todo o dataframe de uma vez


In [ ]:
# Preenchendo todos os dados faltantes com 0
preenchido = dados.fillna(0)

In [ ]:
preenchido

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,0,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0.0,1,2,W./C. 6607,23.4500,0,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# Note que não teremos mais valores nulos no dataset
contagem_nulos = preenchido.isnull().sum()
contagem_nulos

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Substituição pela Média / Item Frequente

In [ ]:
dados_teste = {
    "Produto": ["A", "B", "C", "D"],
    "Quantidade": [10, None, 15, 20],
    "Preco": [100, 200, None, 100]
}

df_vendas = pd.DataFrame(dados_teste)
df_vendas

,Produto,Quantidade,Preco
0,A,10.0,100.0
1,B,NaN,200.0
2,C,15.0,NaN
3,D,20.0,100.0


In [ ]:
# Definindo o valor padrão -1 para a coluna quantidade
df_vendas.fillna({"Quantidade": -1}, inplace=True) ## inplace=True faz com que a operação seja realizada na própria variável

# Obtendo a moda do atributo preço
moda_preco = df_vendas["Preco"].mode()[0]
df_vendas.fillna({'Preco': moda_preco}, inplace=True)

df_vendas

,Produto,Quantidade,Preco
0,A,10.0,100.0
1,B,-1.0,200.0
2,C,15.0,100.0
3,D,20.0,100.0


**EXERCÍCIO 4**: Preencha os dados ausentes do dataset Titanic da seguinte forma:
* Atribua o valor ‘A101’ para as cabines faltantes
* Preencha o atributo idade com o valor médio das idades dos passageiros


In [ ]:
print("Antes:")
contagem_nulos = dados.isnull().sum()
print(contagem_nulos)

preenchidos = dados.copy()
preenchidos.fillna({'Age': preenchidos['Age'].mean()}, inplace=True)
preenchidos.fillna({'Cabin': 'A101'}, inplace=True)

print("\nDepois:")
contagem_nulos = preenchidos.isnull().sum()
print(contagem_nulos)

Antes:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Depois:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64


É possível também usar os valores que estão nas linhas vizinhas para completar os valores faltantes.

**EXERCÍCIO 5**: Procure na documentação da função `fillna` quais as configurações possíveis para o atributo `method`. Crie um exemplo que use esse método para preencher os valores da coluna `Cabin`. Confira o resultado para ver se ele fica conforme o esperado.

In [ ]:
# Preenchendo valores de 'Cabin' com o valor anterior (forward fill)
# Foi de 687 nulos para 1 nulo (O primeiro valor da coluna Cabin é NaN, então não tem de onde puxar o valor anterior)
print("Antes:")
contagem_nulos = dados.isnull().sum()
print(contagem_nulos)

data = dados.copy()
data['Cabin'] = data['Cabin'].fillna(method='ffill')

print("\nDepois:")
contagem_nulos = data.isnull().sum()
print(contagem_nulos)

Antes:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Depois:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            1
Embarked         2
dtype: int64


C:\Users\Ovatman\AppData\Local\Temp\ipykernel_31836\4196703649.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Cabin'] = data['Cabin'].fillna(method='ffill')


### Substituição por Interpolação

Nesse tipo de inputação, os dados são imputados segundo um modelo de interpolação matemática.

In [ ]:
res = dados["Age"].interpolate(method="linear")

for antigo, novo in zip(dados["Age"], res):
    print(antigo, novo)

22.0 22.0
38.0 38.0
26.0 26.0
35.0 35.0
35.0 35.0
nan 44.5
54.0 54.0
2.0 2.0
27.0 27.0
14.0 14.0
4.0 4.0
58.0 58.0
20.0 20.0
39.0 39.0
14.0 14.0
55.0 55.0
2.0 2.0
nan 16.5
31.0 31.0
nan 33.0
35.0 35.0
34.0 34.0
15.0 15.0
28.0 28.0
8.0 8.0
38.0 38.0
nan 28.5
19.0 19.0
nan 26.0
nan 33.0
40.0 40.0
nan 48.666666666666664
nan 57.33333333333333
66.0 66.0
28.0 28.0
42.0 42.0
nan 31.5
21.0 21.0
18.0 18.0
14.0 14.0
40.0 40.0
27.0 27.0
nan 15.0
3.0 3.0
19.0 19.0
nan 18.8
nan 18.6
nan 18.4
nan 18.2
18.0 18.0
7.0 7.0
21.0 21.0
49.0 49.0
29.0 29.0
65.0 65.0
nan 43.0
21.0 21.0
28.5 28.5
5.0 5.0
11.0 11.0
22.0 22.0
38.0 38.0
45.0 45.0
4.0 4.0
nan 12.333333333333334
nan 20.666666666666668
29.0 29.0
19.0 19.0
17.0 17.0
26.0 26.0
32.0 32.0
16.0 16.0
21.0 21.0
26.0 26.0
32.0 32.0
25.0 25.0
nan 16.943333333333335
nan 8.886666666666667
0.83 0.83
30.0 30.0
22.0 22.0
29.0 29.0
nan 28.5
28.0 28.0
17.0 17.0
33.0 33.0
16.0 16.0
nan 19.5
23.0 23.0
24.0 24.0
29.0 29.0
20.0 20.0
46.0 46.0
26.0 26.0
59.0 59.0
nan 6

**Exercício 6**: Faz sentido utilizar interpolação para imputar valores para o atributo idade do dataset de testes? Em que casos isso faz mais sentido?

Faz sentido quando os dados estão em ordem lógica ou temporal, que não é o caso. No dataset do titanic a ordem das linhas não segue uma linha temporal ou lógica para idade.